In [ ]:
!pip install -q datasets accelerate evaluate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.7 MB/s eta 0:00:00
ERROR: Operation c

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import wandb
wandb.login()

ModuleNotFoundError: No module named 'wandb'

In [ ]:
%env WANDB_PROJECT=mamba-vs-gpt2

env: WANDB_PROJECT=gpt2-text-classification
env: WANDB_LOG_MODEL=checkpoint


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from datasets import load_dataset

# Tải dữ liệu IMDB
dataset = load_dataset('imdb')
train_data = dataset['train'].shuffle(seed=1012)
test_data = dataset['test'].shuffle(seed=1012)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
labels = train_data.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
model_name_or_path = "gpt2"

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples['text'],truncation=True,  padding="max_length",max_length=512)

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path, num_labels=2, id2label=id2label, label2id=label2id
)
model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Metrics

In [ ]:
import evaluate
metrics = evaluate.combine(
    ["accuracy", "f1","recall","precision"], force_prefix=False
)
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    outputs = metrics.compute(predictions=predictions,references=labels)
    return outputs

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    f"gpt2-text-classification-v2",
    num_train_epochs=10,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size= 32,
    gradient_accumulation_steps=3,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=True,
    report_to="wandb",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    compute_metrics=compute_metrics,
    eval_dataset=test_data,
    data_collator=data_collator,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01)]
)


In [ ]:
results = trainer.train(resume_from_checkpoint=True)
wandb.finish()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
6,0.112000,0.209933,0.931520,0.930287,0.913840,0.947338
7,0.090300,0.200177,0.934200,0.934013,0.931360,0.936680


wandb: Adding directory to artifact (./gpt2-text-classification-v2/checkpoint-1820)... Done. 6.1s
wandb: Adding directory to artifact (./gpt2-text-classification-v2/checkpoint-2081)... Done. 4.1s


eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,█▁
eval/recall,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▄▄▂█▅▄▂▂▃▃▄▃▂▂▆▄▄▆▁▄▂▂▂▇▁▁▃▂▂▂▄▆▃▂▂▃▄▄▁▃


In [ ]:
results

TrainOutput(global_step=2081, training_loss=0.026607572813981545, metrics={'train_runtime': 1649.7063, 'train_samples_per_second': 151.542, 'train_steps_per_second': 1.576, 'total_flos': 5.215901216617267e+16, 'train_loss': 0.026607572813981545, 'epoch': 7.998721227621483})

In [ ]:
trainer.save_model()
tokenizer.save_pretrained("date3k2/gpt2-text-classification-v2")

('date3k2/gpt2-text-classification-v2/tokenizer_config.json',
 'date3k2/gpt2-text-classification-v2/special_tokens_map.json',
 'date3k2/gpt2-text-classification-v2/vocab.json',
 'date3k2/gpt2-text-classification-v2/merges.txt',
 'date3k2/gpt2-text-classification-v2/added_tokens.json')

In [ ]:
trainer.push_to_hub()

NameError: name 'trainer' is not defined

In [ ]:
tokenizer.push_to_hub("date3k2/gpt2-text-classification-v2")

README.md:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/date3k2/gpt2-text-classification-v2/commit/60ada4244bfea68315977273478424d7a69685a2', commit_message='Upload tokenizer', commit_description='', oid='60ada4244bfea68315977273478424d7a69685a2', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "date3k2/mistral-v0.3-alpaca-vi-v1"
model = AutoModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", load_in_8bit=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: date3k2/mistral-v0.3-alpaca-vi-v1 does not appear to have a file named config.json. Checkout 'https://huggingface.co/date3k2/mistral-v0.3-alpaca-vi-v1/tree/main' for available files.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
peft_model_id = "date3k2/mistral-v0.3-alpaca-vi-v1"

model = AutoModelForCausalLM.from_pretrained(model_id,device_map="auto", load_in_8bit=True)
model.load_adapter(peft_model_id)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`